# Uses data from both images and a flat csv file to classify samples
### Loads data using loader.py

@author: pawel@ksasprowski.pl

In [1]:
    
import loader

from sklearn.metrics.classification import classification_report, accuracy_score, cohen_kappa_score
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing.label import LabelBinarizer
from tensorflow.python.keras.models import Model

import numpy as np
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Sequential

C:\programs\miniconda3\envs\deep\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\programs\miniconda3\envs\deep\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


### Load data

In [2]:
samplesIMG,labels,namesIMG = loader.load_img("radio_img")
samplesCSV,labelsCSV,namesCSV = loader.load_csv("radio.csv")

### Find pairs for samplesIMG in samplesCSV

In [3]:
samples_paired = []
for i in range(samplesIMG.shape[0]):
    for j in range(samplesCSV.shape[0]):
        if namesCSV[j]==namesIMG[i]:
            samples_paired.append(samplesCSV[j])
            
samplesCSV = np.array(samples_paired)
samplesIMG = np.expand_dims(samplesIMG, axis=3)

print("Paired")
print("Samples IMG: {}".format(len(samplesIMG)))
print("Samples CSV: {}".format(len(samplesCSV)))

Paired
Samples IMG: 611
Samples CSV: 611


### One-hot encoding

In [4]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

### Shapes

In [5]:
numClasses = labels.shape[1]
inputShape = (108,192,1) #samplesIMG.shape

### Model for images

In [6]:
cnnmodel = Sequential()
cnnmodel.add(Conv2D(16, (3, 3), padding="same",input_shape=inputShape))
cnnmodel.add(Activation("relu"))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Conv2D(32, (3, 3), padding="same"))
cnnmodel.add(Activation("relu"))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Dropout(0.25))
cnnmodel.add(Flatten())
cnnmodel.add(Dense(16))
cnnmodel.add(Activation("relu"))

### Model for features (CSV)

In [7]:
flatmodel = Sequential()
flatmodel.add(Flatten(input_shape=(14,)))
flatmodel.add(Dense(50, activation='sigmoid'))

### Concatenated model

In [8]:
combined = concatenate([cnnmodel.output, flatmodel.output])
combined = Dense(16, activation="sigmoid")(combined)
combined = Dense(numClasses, activation="sigmoid")(combined)

model = Model(inputs=[cnnmodel.input, flatmodel.input], outputs=combined)

print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 108, 192, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 108, 192, 16) 160         conv2d_input[0][0]               
__________________________________________________________________________________________________
activation (Activation)         (None, 108, 192, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 54, 96, 16)   0           activation[0][0]                 
_______________________________________________________________________________________

### Training

In [9]:
EPOCHS = 10
BATCH=100
model.fit([samplesIMG,samplesCSV], labels, batch_size=BATCH, epochs=EPOCHS, 
          validation_data=[[samplesIMG,samplesCSV],labels])

Epoch 1/10
7/7 [==============================] - 5s 725ms/step - loss: 1.0556 - accuracy: 0.3977 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 6s 800ms/step - loss: 0.9631 - accuracy: 0.5957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
7/7 [==============================] - 5s 752ms/step - loss: 0.9224 - accuracy: 0.5941 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
7/7 [==============================] - 5s 685ms/step - loss: 0.9029 - accuracy: 0.6219 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
7/7 [==============================] - 5s 688ms/step - loss: 0.8667 - accuracy: 0.6268 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
7/7 [==============================] - 5s 703ms/step - loss: 0.8246 - accuracy: 0.6252 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
7/7 [==============================] - 6s 807ms/step - loss: 0.7919 - accuracy: 0.6252 - val_loss: 0.

### Results

In [10]:
results = model.predict([samplesIMG,samplesCSV])
print(confusion_matrix(labels.argmax(axis=1), results.argmax(axis=1)))
print(classification_report(labels.argmax(axis=1), results.argmax(axis=1)))
print("Accuracy: {:.2f}".format(accuracy_score(labels.argmax(axis=1), results.argmax(axis=1))))
print("Cohen's Kappa {:.2f}".format(cohen_kappa_score(labels.argmax(axis=1), results.argmax(axis=1))))

[[ 98   0  26]
 [  1   0 180]
 [  3   0 303]]
              precision    recall  f1-score   support

           0       0.96      0.79      0.87       124
           1       0.00      0.00      0.00       181
           2       0.60      0.99      0.74       306

    accuracy                           0.66       611
   macro avg       0.52      0.59      0.54       611
weighted avg       0.49      0.66      0.55       611

Accuracy: 0.66
Cohen's Kappa 0.37


C:\programs\miniconda3\envs\deep\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
